In [ ]:
import sys
sys.path.append("../notebooks")  # needed to make kfto_sdk_mnist
from kfto_sdk_mnist import train_func
from kubeflow.training import TrainingClient
from kubernetes import client as c
import time

In [2]:
# parameters
num_gpus = "${num_gpus}"
openshift_api_url = "${api_url}"
namespace = "${namespace}"
token = "${password}"

In [ ]:
config = c.Configuration(host=openshift_api_url, api_key=token)
config.verify_ssl = False
tc = TrainingClient()

In [ ]:
try:
   tc.create_job(
      name="pytorch-ddp",
      namespace=namespace,
      train_func=train_func,
      num_workers=2,
      resources_per_worker={"gpu": num_gpus},
      base_image="quay.io/kpostlet/torch-train:withvision",
   )
except:
   pass

In [ ]:
while not tc.is_job_succeeded(name="pytorch-ddp", namespace=namespace): 
    time.sleep(1)

In [ ]:
TrainingClient().delete_job(
    name="pytorch-ddp",
    namespace=namespace,
)